In [1]:
import sys, os
# sys.path.insert(0, "/home/martin/workspace/LightCurvesClassifier/LightCurvesClassifier")
import pandas as pd


from lcc.db_tier.stars_provider import StarsProvider
from lcc.stars_processing.tools.params_estim import ParamsEstimator
from lcc.api.input_parse import parse_tun_query
from lcc.utils.helpers import  get_combinations
from lcc.data_manager.package_reader import PackageReader
from lcc.stars_processing.systematic_search.stars_searcher import StarsSearcher
from lcc.stars_processing.tools.visualization import plotProbabSpace

In [2]:
# The query #
#=============

# Descriptor and decider
descr_name1 = "AbbeValueDescr"
descr_name2 = "VariogramSlopeDescr"
decid_name = "GaussianNBDec"


# Tunning parameters
tun_param1 = "bins"
bin_from = 10
bin_to = 150
bin_step = 50

tun_param2 = "days_per_bin"
dpb_from = 30
dpb_to = 110
dpb_step = 10


# Loading training stars
LCS_PATH = "/home/martin/workspace/LightCurvesClassifier/data/project/inp_lcs"
obt_method = "FileManager"
quasars_path = os.path.join(LCS_PATH, "quasars")
stars_path = os.path.join(LCS_PATH, "some_stars")

# Query for OgleII
db_name = "OgleII"
starid_from = 1
starid_to = 10
field_num_from = 1
field_num_to = 2
target = "lmc"

In [3]:
# Prepare for tuning
abbe_descr = PackageReader().getClassesDict("descriptors").get(descr_name1)
vario_slope_descr = PackageReader().getClassesDict("descriptors").get(descr_name2)
decider = PackageReader().getClassesDict("deciders").get(decid_name)


combinations = get_combinations([":".join([descr_name1, tun_param1]), ":".join([descr_name2, tun_param2])],
                                 range(bin_from, bin_to, bin_step), range(dpb_from, dpb_to, dpb_step))
tun_params = parse_tun_query(combinations)

quasars = StarsProvider.getProvider(obt_method, {"path" : quasars_path}).getStars()
stars = StarsProvider.getProvider(obt_method, {"path" : stars_path}).getStars()

INFO:root:No fast networks available: No module named arac.pybrainbridge


Loading dat files:[############################################################] 44/44
Loading dat files:[############################################################] 237/237


In [6]:
# Estimate all combinations and get the best one
es = ParamsEstimator(searched=quasars,
                     others=stars,
                     descriptors=[abbe_descr, vario_slope_descr],
                     deciders=[decider],
                     tuned_params=tun_params)

star_filter, best_stats, best_params = es.fit()

All 24 combinations have been evaluated

In [7]:
star_filter.evaluateStars(quasars[:10])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [ ]:
star_filter.searched_coords.head()

,Abbe value,Light curve's variogram slope
LMC_SC586317,0.168439,0.925078
LMC_SC7346891,0.546002,0.998490
LMC_SC21116867,0.156445,1.674246
SMC_SC4146087,0.305307,1.952205
LMC_SC19252805,0.411018,0.967198


In [ ]:
plotProbabSpace(star_filter)

In [ ]:
from lcc.utils.output_process_modules import saveIntoFile, loadFromFile
saveIntoFile(star_filter, "/tmp", "my_filter.obj")

In [ ]:
my_filter = loadFromFile("/tmp/my_filter.obj")

In [ ]:
star_filter.searched_coords

In [ ]:
# Prepare queries and run systematic search by using the filter
queries = get_combinations(["starid", "field_num", "target"],
                           range(starid_from, starid_to),
                           range(field_num_from, field_num_to),
                           [target])

searcher = StarsSearcher([star_filter],
                         obth_method=db_name)
searcher.queryStars(queries)

passed_stars = searcher.passed_stars

In [ ]:
print "All parameters (the best is %i):\n\n%s\n\n" % (es.best_id, pd.DataFrame(es.stats))
print "Searching result (passed %i from %i) \n\n%s" % (len(passed_stars), len(searcher.status),searcher.status)

In [ ]:
from lcc.utils.stars import saveStars
saveStars(quasars[:5], "/tmp")

In [ ]:
quasars[0].lightCurve.meta